In [1]:
import json
import time
import cv2
import numpy as np
import pymongo
from scipy.spatial import KDTree
import os
import seaborn as sn
import pandas as pd

In [2]:
class EuclideanDistance:
    def __init__(self) -> None:
        pass
    def calculate_distance(self, x, y):
        return np.linalg.norm(x-y)
    
class Matcher:
    def __init__(self, *args, features_name=[], extractors=[], collection=None, metric=None, **kwargs):
        self.features_name = features_name
        self.extractors = extractors
        self.collection = collection
        self.metric = metric

    def get_features(self, image):
        features = [extractor.extract(image) for extractor in self.extractors]
        features = np.concatenate(features)
        return features

    def match(self, image, *args, ntop=10, **kwargs):
        pass

class ExhaustiveMatcher(Matcher):
    def __init__(self, *args, features_name=[], extractors=[], collection=None, metric=None, **kwargs):
        super().__init__(*args, features_name=features_name, extractors=extractors, collection=collection, metric=metric, **kwargs)
    
    def match(self, image, *args, ntop=10, **kwargs):
        features = [extractors.extract(image)]
        features = np.concatenate(features)
        res = []
        for record in self.collection:
            record_features = record[1]
            distance = self.metric.calculate_distance(features, record_features)
            res.append((record[0], distance))
            idx = len(res) - 1
            while(idx > 0) and (distance < res[idx-1][1]):
                res[idx], res[idx-1] = res[idx-1], res[idx]
                idx -= 1
            if len(res) > ntop:
                res.pop(-1)
        return res

class KDTreeMatcher(Matcher):
    def __init__(self, *args, features_name=[], extractors=[], collection=None, metric=None, **kwargs):
        super().__init__(*args, features_name=features_name, extractors=extractors, collection=collection, metric=metric **kwargs)
        self.kd_tree = KDTree([record['features']['SIFT'] for record in collection])

    def match(self, image, *args, ntop=10, **kwargs):
        features = self.get_features(image)
        dd, ii = self.kd_tree.query([features], k=ntop)
        dd = dd[0]
        ii = ii[0]
        res = [(self.collection[ii[i]], dd[i]) for i in range(len(ii))]
        return res

In [ ]:
import cv2
import numpy as np
from numpy.core.fromnumeric import squeeze


class FeatureExtractor:
    def __init__(self, *args, **kwargs):
        pass
    
    def extract(self, image, *args, **kwargs):
        raise Exception("extract function must be implemented")

class Random(FeatureExtractor):
    def __init__(self, *args, feature_size=16, **kwargs):
        super().__init__(*args, **kwargs)
        self.feature_size = feature_size

    def extract(self, image, *args, **kwargs):
        return np.random.rand(*self.feature_size)

class HuMoments(FeatureExtractor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def extract(self, image, *args, **kwargs):
        shape = image.shape
        if len(shape) > 2:
            image = np.average(image, axis=tuple(range(2, len(shape))))
        moments = cv2.moments(image)
        hu_moments = cv2.HuMoments(moments)
        hu_moments = np.squeeze(hu_moments)
        log_hu_moments = -1 * np.copysign(1.0, hu_moments) * np.log10(np.abs(hu_moments))
        return log_hu_moments

class SIFT(FeatureExtractor):
    def __init__(self, *arg, **kwargs):
        super().__init__(*arg, **kwargs)
        self.extractor = cv2.SIFT_create()
        self.eps = 1e-7
        self.isRootSIFT = False
        self.size = 1024

    def extract(self, image, *args, **kwargs):
        kp, descriptor = self.extract_full(image, *args, **kwargs)
        kp_des = [(kp[i], descriptor[i]) for i in range(len(kp))]
        kp_des.sort(key=lambda x: x[0].response, reverse=True)
        if len(kp_des) > 0:
            features = np.concatenate([d[1] for d in kp_des])
            if features.shape[0] < 1024:
                features = np.concatenate([features, np.zeros(1024 - features.shape[0])])
        else:
            features = np.zeros(1024)
        return features[:1024]
    
    def extract_full(self, image, *args, **kwargs):
        kp, descriptor = self.extractor.detectAndCompute(image, None)
        if self.isRootSIFT == True:
            descriptor /= (descriptor.sum(axis=1, keepdims=True) + self.eps)
            descriptor = np.sqrt(descriptor)
        return kp, descriptor

class HOG(FeatureExtractor):
    def __init__(self, *args, winSize=(32, 32), blockSize=(32, 32), blockStride=(2, 2), cellSize=(16, 16), nbins=9, **kwargs):
        super().__init__(*args, **kwargs)
        self.winSize = winSize # Image size
        self.blockSize = blockSize # multiple of cell size, for histogram normalization
        self.blockStride = blockStride # block overlapping
        self.cellSize = cellSize # each cell has 1 histogram
        self.nbins = nbins # number of directions
        self.extractor = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    
    def extract(self, image, *args, **kwargs):
        features = self.extractor.compute(image)
        features = np.squeeze(features)
        return features

class ColorHistogram(FeatureExtractor):
    def __init__(self, *args, nbins = 8, type='HSV', **kwargs):
        super().__init__(*args, **kwargs)
        self.nbins = nbins
        self.type = type
    
    def extract(self, image, *args, **kwargs):
        if type == 'HSV':
            # convert the image from BGR to HSV  format
            image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            # compute the color histograms
            histograms  = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
            # normalize the histograms
            cv2.normalize(histograms, histograms)
            # return the histograms
            return histograms.flatten()
        elif type == 'RGB':
            b, g, r = cv2.split(image)
            # compute the color histograms
            rgb_hist = np.zeros((768,1), dtype='uint32')
            b_hist = cv2.calcHist([b], [0], None, [256], [0, 256])
            g_hist = cv2.calcHist([g], [0], None, [256], [0, 256])
            r_hist = cv2.calcHist([r], [0], None, [256], [0, 256])
            rgb_hist = np.array([r_hist, g_hist, b_hist])
            # normalize the histograms
            cv2.normalize(rgb_hist, rgb_hist)
            # return the histograms
            return rgb_hist.flatten()
        return np.zeros(self.nbins * 3)


In [4]:
# Thay doi ham nay de xu ly rieng tung dataset, tung extractor
def read_image_from_config(config, dataset=None, extractor=None):
    image_path = config['image_path']

    # Voi dataset la coil-100
    if dataset == 'coil-100':
        if extractor == 'HuMoments':
            image = cv2.imread(image_path, 0)
            image = cv2.GaussianBlur(image, (5,5), 0)
            return image
        elif extractor == 'HOG':
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            blur = cv2.bilateralFilter(image, 9, 75, 75)
            return image
        elif extractor == 'HOG_HSV':
            image = cv2.imread(image_path)
            hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            hue, _, _ = cv2.split(hsv_img)
            blur = cv2.bilateralFilter(hue, 9, 75, 75)
            return image
        elif extractor == 'SIFT':
            image = cv2.imread(image_path, 0)
            return image
        elif extractor == 'ColorHistogram':
            return cv2.imread(image_path)
        else:
            return cv2.imread(image_path, 0)

    # Voi dataset la caltech-101
    elif dataset == 'caltech-101':
        if extractor == 'HuMoments':
            image = cv2.imread(image_path, 0)
            blur = cv2.GaussianBlur(image, (5,5), 0)
            return blur
        elif extractor == 'HOG':
            image = cv2.imread(image_path, 0)
            blur = cv2.bilateralFilter(image,9,75,75)
            padding = add_padding(blur)
            resize = cv2.resize(padding, (128, 128), interpolation=cv2.INTER_AREA)
            return resize
        elif extractor == 'HOG_HSV':
            image = cv2.imread(image_path)
            hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            hue, _, _ = cv2.split(hsv_img)
            blur = cv2.bilateralFilter(hue, 9, 75, 75)
            return blur
        elif extractor == 'SIFT':
            return cv2.imread(image_path, 0)
        elif extractor == 'ColorHistogram':
            return cv2.imread(image_path)

    elif dataset == 'cifar-10':
        if extractor == 'HOG':
            image = cv2.imread(image_path, 0)
        else:
            return cv2.imread(image_path, 0)
    else:
        return cv2.imread(image_path, 0)



In [5]:
from feature_extractor import *

# Test với HuMoments
list_features={'HuMoments'}

coil100 = './data/coil-100'

dataset_path = coil100
ds = dataset_path.split('/')[-1]

trainset_path = dataset_path + '/train.json'
testset_path = dataset_path + '/test.json'

trainset_des = list()
testset_des = list()

with open(trainset_path) as f:
    trainset_des = json.load(f)
with open(testset_path) as f:
    testset_des = json.load(f)
    
print(len(trainset_des))
print(len(testset_des))


extractors = HuMoments()
train_collection = list()
test_collection = list()
start_time = time.time()
for des in trainset_des:
    image = read_image_from_config(des, dataset=ds, extractor='HuMoments')
    feature = [extractors.extract(image)]
    feature = np.concatenate(feature)
    train_collection.append((des, feature))
print("Train set is done !!!")
    
for des in testset_des:
    image = read_image_from_config(des, dataset=ds, extractor='HuMoments')
    feature = [extractors.extract(image)]
    feature = np.concatenate(feature)
    test_collection.append((des, feature))
print("Test set is done !!!")
print('Time:', time.time() - start_time)

metric = EuclideanDistance()
matcher = ExhaustiveMatcher(features_name=list_features, extractors=extractors, collection=train_collection, metric=metric)

all_classes = os.listdir(dataset_path + '/train')
num_classes = len(all_classes)

list_n_top = [1, 3, 5, 10]
count_success = dict()
count = dict()
total = dict()
confusion_matrix = dict()
for n_top in list_n_top:
    count_success[n_top] = count[n_top] = total[n_top] = 0
    confusion_matrix[n_top] = dict()
    for c1 in all_classes:
        confusion_matrix[n_top][c1] = dict()
        for c2 in all_classes:
            confusion_matrix[n_top][c1][c2] = 0
    count[n_top] = total[n_top] = 0
sample_count = 0

for des in testset_des:
    image = read_image_from_config(des, dataset=ds, extractor='HuMoments')
    res = matcher.match(image)
    image_class_name = des['class_name']

    for n_top in list_n_top:
        records_class_name = [r[0]['class_name'] for r in res[:n_top]]
        for record_class_name in records_class_name:
            confusion_matrix[n_top][image_class_name][record_class_name] += 1
            if image_class_name == record_class_name:
                count[n_top] += 1
        if image_class_name in records_class_name:
            count_success[n_top] += 1
        total[n_top] += len(records_class_name)

    sample_count += 1
    if sample_count % 100 == 0:
        msg = []
        msg2 = []
        for n_top in list_n_top:
            msg.append(' '.join(['%d:' % (n_top,), "%.2f" % (count[n_top] * 100 / total[n_top]) + '%']))
            msg2.append(' '.join(['%d:' % (n_top,), "%.2f" % (count_success[n_top] * 100 / sample_count) + '%']))
        print(sample_count, 'accuracy' + '; '.join(msg), 'success:' + '; '.join(msg2))
    # if sample_count == 100:
    #     break

for n_top in list_n_top:
    print('Top %d accuracy:' % (n_top,), str(count[n_top] * 100 / total[n_top]) + '%')
    print('Top %d success:' % (n_top,), str(count_success[n_top] * 100 / sample_count) + '%')
    
n_top = 10
print()
    
# cf = np.zeros((num_classes, num_classes), dtype='uint8')
# for i in range(num_classes):
#     for j in range(num_classes):
#         cf[i,j] = confusion_matrix[n_top][all_classes[j]][all_classes[i]]
# plot_confusion_matrix(cf, classes=all_classes, output_file=dataset+'_'+list(list_features)[0] + '.png')

5700
1500
Train set is done !!!
Test set is done !!!
Time: 10.198000192642212
100 accuracy1: 61.00%; 3: 47.67%; 5: 41.20%; 10: 31.70% success:1: 61.00%; 3: 78.00%; 5: 84.00%; 10: 91.00%
200 accuracy1: 65.50%; 3: 54.83%; 5: 50.10%; 10: 41.35% success:1: 65.50%; 3: 76.50%; 5: 84.50%; 10: 89.00%
300 accuracy1: 72.67%; 3: 63.00%; 5: 57.33%; 10: 48.53% success:1: 72.67%; 3: 82.33%; 5: 88.33%; 10: 91.33%
400 accuracy1: 72.75%; 3: 61.25%; 5: 55.30%; 10: 46.42% success:1: 72.75%; 3: 81.25%; 5: 86.50%; 10: 90.00%
500 accuracy1: 74.00%; 3: 63.07%; 5: 57.20%; 10: 48.34% success:1: 74.00%; 3: 82.20%; 5: 86.80%; 10: 90.40%
600 accuracy1: 73.00%; 3: 62.17%; 5: 55.73%; 10: 46.23% success:1: 73.00%; 3: 82.17%; 5: 86.67%; 10: 90.17%
700 accuracy1: 74.43%; 3: 63.76%; 5: 57.29%; 10: 47.37% success:1: 74.43%; 3: 83.29%; 5: 87.43%; 10: 91.00%
800 accuracy1: 74.88%; 3: 64.42%; 5: 57.92%; 10: 48.24% success:1: 74.88%; 3: 83.38%; 5: 87.38%; 10: 91.50%
900 accuracy1: 73.67%; 3: 62.93%; 5: 56.24%; 10: 46.57% su

In [9]:
# Test với HOG
list_features={'HOG'}

coil100 = './data/coil-100'

dataset_path = coil100

trainset_path = dataset_path + '/train.json'
testset_path = dataset_path + '/test.json'

trainset_des = list()
testset_des = list()

with open(trainset_path) as f:
    trainset_des = json.load(f)
with open(testset_path) as f:
    testset_des = json.load(f)
    
print(len(trainset_des))
print(len(testset_des))



from feature_extractor import *
extractors = HOG(winSize=(128, 128), blockSize=(32, 32), blockStride=(16, 16), cellSize=(16, 16))

train_collection = list()
test_collection = list()

start_time = time.time()
for des in trainset_des:
    image = read_image_from_config(des, dataset=ds, extractor='HOG')
    feature = [extractors.extract(image)]
    feature = np.concatenate(feature)
    train_collection.append((des, feature))
print("Train set is done !!!")
    
for des in testset_des:
    image = read_image_from_config(des, dataset=ds, extractor='HOG')
    feature = [extractors.extract(image)]
    feature = np.concatenate(feature)
    test_collection.append((des, feature))
print("Test set is done !!!")
print('Time:', time.time() - start_time)

metric = EuclideanDistance()
matcher = ExhaustiveMatcher(features_name=list_features, extractors=extractors, collection=train_collection, metric=metric)

all_classes = os.listdir(dataset_path + '/train')
num_classes = len(all_classes)

list_n_top = [1, 3, 5, 10]
count_success = dict()
count = dict()
total = dict()
confusion_matrix = dict()
for n_top in list_n_top:
    count_success[n_top] = count[n_top] = total[n_top] = 0
    confusion_matrix[n_top] = dict()
    for c1 in all_classes:
        confusion_matrix[n_top][c1] = dict()
        for c2 in all_classes:
            confusion_matrix[n_top][c1][c2] = 0
    count[n_top] = total[n_top] = 0
sample_count = 0

for des in testset_des:
    image = read_image_from_config(des, dataset=ds, extractor='HOG')
    res = matcher.match(image)
    image_class_name = des['class_name']

    for n_top in list_n_top:
        records_class_name = [r[0]['class_name'] for r in res[:n_top]]
        for record_class_name in records_class_name:
            confusion_matrix[n_top][image_class_name][record_class_name] += 1
            if image_class_name == record_class_name:
                count[n_top] += 1
        if image_class_name in records_class_name:
            count_success[n_top] += 1
        total[n_top] += len(records_class_name)

    sample_count += 1
    if sample_count % 100 == 0:
        msg = []
        msg2 = []
        for n_top in list_n_top:
            msg.append(' '.join(['%d:' % (n_top,), "%.2f" % (count[n_top] * 100 / total[n_top]) + '%']))
            msg2.append(' '.join(['%d:' % (n_top,), "%.2f" % (count_success[n_top] * 100 / sample_count) + '%']))
        print(sample_count, 'accuracy' + '; '.join(msg), 'success:' + '; '.join(msg2))
    # if sample_count == 100:
    #     break

for n_top in list_n_top:
    print('Top %d accuracy:' % (n_top,), str(count[n_top] * 100 / total[n_top]) + '%')
    print('Top %d success:' % (n_top,), str(count_success[n_top] * 100 / sample_count) + '%')

5700
1500
Train set is done !!!
Test set is done !!!
Time: 14.197999954223633
100 accuracy1: 98.00%; 3: 96.33%; 5: 92.00%; 10: 80.60% success:1: 98.00%; 3: 99.00%; 5: 99.00%; 10: 99.00%
200 accuracy1: 97.50%; 3: 94.50%; 5: 90.40%; 10: 79.75% success:1: 97.50%; 3: 98.50%; 5: 98.50%; 10: 99.00%
300 accuracy1: 97.67%; 3: 94.00%; 5: 89.53%; 10: 80.00% success:1: 97.67%; 3: 98.33%; 5: 98.33%; 10: 99.00%
400 accuracy1: 97.50%; 3: 94.08%; 5: 87.60%; 10: 76.08% success:1: 97.50%; 3: 98.50%; 5: 98.50%; 10: 99.00%
500 accuracy1: 98.00%; 3: 94.53%; 5: 88.68%; 10: 77.90% success:1: 98.00%; 3: 98.80%; 5: 98.80%; 10: 99.20%
600 accuracy1: 98.17%; 3: 94.28%; 5: 87.83%; 10: 76.20% success:1: 98.17%; 3: 98.83%; 5: 98.83%; 10: 99.33%
700 accuracy1: 98.43%; 3: 94.76%; 5: 88.60%; 10: 77.63% success:1: 98.43%; 3: 99.00%; 5: 99.00%; 10: 99.43%
800 accuracy1: 98.62%; 3: 95.04%; 5: 89.28%; 10: 78.85% success:1: 98.62%; 3: 99.12%; 5: 99.12%; 10: 99.50%
900 accuracy1: 98.78%; 3: 94.63%; 5: 88.47%; 10: 77.28% su

In [10]:
# Test với HOG
list_features={'HOG'}

coil100 = './data/coil-100'

dataset_path = coil100

trainset_path = dataset_path + '/train.json'
testset_path = dataset_path + '/test.json'

trainset_des = list()
testset_des = list()

with open(trainset_path) as f:
    trainset_des = json.load(f)
with open(testset_path) as f:
    testset_des = json.load(f)
    
print(len(trainset_des))
print(len(testset_des))


from feature_extractor import *
extractors = ColorHistogram()

train_collection = list()
test_collection = list()

start_time = time.time()
for des in trainset_des:
    image = read_image_from_config(des, dataset=ds, extractor='ColorHistogram')
    feature = [extractors.extract(image)]
    feature = np.concatenate(feature)
    train_collection.append((des, feature))
print("Train set is done !!!")
    
for des in testset_des:
    image = read_image_from_config(des,  dataset=ds, extractor='ColorHistogram')
    feature = [extractors.extract(image)]
    feature = np.concatenate(feature)
    test_collection.append((des, feature))
print("Test set is done !!!")
print('Time:', time.time() - start_time)

metric = EuclideanDistance()
matcher = ExhaustiveMatcher(features_name=list_features, extractors=extractors, collection=train_collection, metric=metric)

all_classes = os.listdir(dataset_path + '/train')
num_classes = len(all_classes)

list_n_top = [1, 3, 5, 10]
count_success = dict()
count = dict()
total = dict()
confusion_matrix = dict()
for n_top in list_n_top:
    count_success[n_top] = count[n_top] = total[n_top] = 0
    confusion_matrix[n_top] = dict()
    for c1 in all_classes:
        confusion_matrix[n_top][c1] = dict()
        for c2 in all_classes:
            confusion_matrix[n_top][c1][c2] = 0
    count[n_top] = total[n_top] = 0
sample_count = 0

for des in testset_des:
    image = read_image_from_config(des, dataset=ds, extractor='ColorHistogram')
    res = matcher.match(image)
    image_class_name = des['class_name']

    for n_top in list_n_top:
        records_class_name = [r[0]['class_name'] for r in res[:n_top]]
        for record_class_name in records_class_name:
            confusion_matrix[n_top][image_class_name][record_class_name] += 1
            if image_class_name == record_class_name:
                count[n_top] += 1
        if image_class_name in records_class_name:
            count_success[n_top] += 1
        total[n_top] += len(records_class_name)

    sample_count += 1
    if sample_count % 100 == 0:
        msg = []
        msg2 = []
        for n_top in list_n_top:
            msg.append(' '.join(['%d:' % (n_top,), "%.2f" % (count[n_top] * 100 / total[n_top]) + '%']))
            msg2.append(' '.join(['%d:' % (n_top,), "%.2f" % (count_success[n_top] * 100 / sample_count) + '%']))
        print(sample_count, 'accuracy' + '; '.join(msg), 'success:' + '; '.join(msg2))
    # if sample_count == 100:
    #     break

for n_top in list_n_top:
    print('Top %d accuracy:' % (n_top,), str(count[n_top] * 100 / total[n_top]) + '%')
    print('Top %d success:' % (n_top,), str(count_success[n_top] * 100 / sample_count) + '%')

5700
1500
Train set is done !!!
Test set is done !!!
Time: 7.152998924255371
100 accuracy1: 98.00%; 3: 97.00%; 5: 94.40%; 10: 88.70% success:1: 98.00%; 3: 100.00%; 5: 100.00%; 10: 100.00%
200 accuracy1: 99.00%; 3: 98.00%; 5: 96.30%; 10: 92.45% success:1: 99.00%; 3: 100.00%; 5: 100.00%; 10: 100.00%
300 accuracy1: 99.33%; 3: 98.67%; 5: 97.47%; 10: 94.43% success:1: 99.33%; 3: 100.00%; 5: 100.00%; 10: 100.00%
400 accuracy1: 99.50%; 3: 98.92%; 5: 97.95%; 10: 95.33% success:1: 99.50%; 3: 100.00%; 5: 100.00%; 10: 100.00%
500 accuracy1: 99.60%; 3: 98.80%; 5: 97.80%; 10: 95.40% success:1: 99.60%; 3: 100.00%; 5: 100.00%; 10: 100.00%
600 accuracy1: 99.67%; 3: 99.00%; 5: 98.10%; 10: 95.70% success:1: 99.67%; 3: 100.00%; 5: 100.00%; 10: 100.00%
700 accuracy1: 99.57%; 3: 98.95%; 5: 98.20%; 10: 95.89% success:1: 99.57%; 3: 99.86%; 5: 100.00%; 10: 100.00%
800 accuracy1: 99.50%; 3: 98.54%; 5: 97.45%; 10: 94.92% success:1: 99.50%; 3: 99.88%; 5: 100.00%; 10: 100.00%
900 accuracy1: 99.44%; 3: 98.48%; 5: 

In [ ]:
# Top 1 accuracy: 99.46666666666667%
# Top 3 accuracy: 98.28888888888889%
# Top 5 accuracy: 96.81333333333333%
# Top 10 accuracy: 93.68%
# def fd_histogram(image, mask=None):
#     bins = 8
#     # convert the image from BGR to HSV  format
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     h,s, v = cv2.split(image)
#     b, g, r = cv2.split(image)
#     # compute the color histograms
#     histograms  = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
#     # normalize the histograms
#     cv2.normalize(histograms, histograms)
#     # return the histograms
#     return histograms.flatten()

# Top 1 accuracy: 99.26666666666667%
# Top 3 accuracy: 97.97777777777777%
# Top 5 accuracy: 96.38666666666667%
# Top 10 accuracy: 92.58%
# def fd_histogram(image, mask=None):
#     bins = 8
#     # convert the image from BGR to HSV  format
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     h,s, v = cv2.split(image)
#     b, g, r = cv2.split(image)
#     # compute the color histograms
#     histograms  = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
#     # normalize the histograms
#     cv2.normalize(histograms, histograms)
#     # return the histograms
#     return histograms.flatten()
# Top 1 accuracy: 98.2%
# Top 3 accuracy: 96.28888888888889%
# Top 5 accuracy: 94.76%
# Top 10 accuracy: 90.78666666666666%
# def fd_histogram(image, mask=None):
#     bins = 8
#     # convert the image from BGR to HSV  format
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     h,s, v = cv2.split(image)
#     b, g, r = cv2.split(image)
#     # compute the color histograms
#     histograms  = cv2.calcHist([image], [0, 1, 2], None, [8, 3, 3], [0, 256, 0, 256, 0, 256])
#     # normalize the histograms
#     cv2.normalize(histograms, histograms)
#     # return the histograms
#     return histograms.flatten()

# Top 1 accuracy: 93.2%
# Top 3 accuracy: 87.0%
# Top 5 accuracy: 82.4%
# Top 10 accuracy: 73.58666666666667%
# def fd_histogram(image, mask=None):
#     b, g, r = cv2.split(image)
#     # compute the color histograms
#     # histograms  = cv2.calcHist([image], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
#     rgb_hist = np.zeros((768,1), dtype='uint32')
#     b_hist = cv2.calcHist([b], [0], None, [256], [0, 256])
#     g_hist = cv2.calcHist([g], [0], None, [256], [0, 256])
#     r_hist = cv2.calcHist([r], [0], None, [256], [0, 256])
#     rgb_hist = np.array([r_hist, g_hist, b_hist])
#     # normalize the histograms
#     cv2.normalize(rgb_hist, rgb_hist)
#     # return the histograms
#     return rgb_hist.flatten()

# Top 1 accuracy: 88.6%
# Top 3 accuracy: 80.26666666666667%
# Top 5 accuracy: 75.24%
# Top 10 accuracy: 65.95333333333333%

# def fd_histogram(image):
#     b, g, r = cv2.split(image)
#     # compute the color histograms
#     histograms  = cv2.calcHist([image], [0, 1], None, [180, 256], [0, 180, 0, 256])
#     # normalize the histograms
#     cv2.normalize(histograms, histograms)
#     # return the histograms
#     return histograms.flatten()

# Top 1 accuracy: 98.06666666666666%
# Top 3 accuracy: 93.91111111111111%
# Top 5 accuracy: 89.34666666666666%
# Top 10 accuracy: 79.3%
# def fd_histogram(image):
#     extractors = HOG(winSize=(128, 128), blockSize=(32, 32), blockStride=(16, 16), cellSize=(16, 16))
#     hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     hue, sat, val = cv2.split(hsv_img)
#     blurred = cv2.bilateralFilter(sat, 9, 75, 75)
#     return extractors.extract(blurred)

# Top 1 accuracy: 98.93333333333334%
# Top 3 accuracy: 96.17777777777778%
# Top 5 accuracy: 92.94666666666667%
# Top 10 accuracy: 85.27333333333333%
# def fd_histogram(image):
#     extractors = HOG(winSize=(128, 128), blockSize=(32, 32), blockStride=(16, 16), cellSize=(16, 16))
#     hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     hue, sat, val = cv2.split(hsv_img)
#     blurred = cv2.bilateralFilter(hue, 9, 75, 75)
#     return extractors.extract(blurred)

# Top 1 accuracy: 97.53333333333333%
# Top 3 accuracy: 93.22222222222223%
# Top 5 accuracy: 88.50666666666666%
# Top 10 accuracy: 78.61333333333333%
# def fd_histogram(image):
#     extractors = HOG(winSize=(128, 128), blockSize=(32, 32), blockStride=(16, 16), cellSize=(16, 16))
#     hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     hue, sat, val = cv2.split(hsv_img)
#     blurred = cv2.bilateralFilter(val, 9, 75, 75)
#     return extractors.extract(blurred)

# Top 1 accuracy: 97.93333333333334%
# Top 3 accuracy: 94.62222222222222%
# Top 5 accuracy: 91.14666666666666%
# Top 10 accuracy: 83.52%
# def fd_histogram(image):
#     extractors = HOG(winSize=(128, 128), blockSize=(32, 32), blockStride=(16, 16), cellSize=(16, 16))
#     hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     hue, sat, val = cv2.split(hsv_img)
#     return extractors.extract(hue)


# Top 1 accuracy: 99.26666666666667%
# Top 3 accuracy: 96.95555555555555%
# Top 5 accuracy: 94.01333333333334%
# Top 10 accuracy: 86.54%
# def fd_histogram(image):
#     extractors = HOG(winSize=(64, 64), blockSize=(16, 16), blockStride=(8, 8), cellSize=(8, 8))
#     image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_AREA)
#     hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     hue, sat, val = cv2.split(hsv_img)
#     blurred = cv2.bilateralFilter(hue, 9, 75, 75)
#     return extractors.extract(blurred)